In [2]:
#@title Carregar os dados de fluxo acadêmico do Censo da Educação Superior
!pip install -q gdown
import os
import gdown
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

S_UFC = 'Universidade Federal do Ceará'

# Função para formatar texto com apenas a primeira letra em maiúsculo
def titulo_formatado(texto):
    excecoes = {"de", "da", "do", "das", "dos", "e", "em"}
    texto = texto.replace("(","").replace(")","")
    palavras = texto.lower().split()

    resultado = []
    for i, palavra in enumerate(palavras):
        if palavra in excecoes and i != 0:
            resultado.append(palavra)
        else:
            resultado.append(palavra.capitalize())

    return ' '.join(resultado)

# Carrega dados dos códigos dos estados
id_google = '1qosijWRFUd32b_0erehxdXq2a-W5NYg5'
url = f"https://drive.google.com/uc?id={id_google}&confirm=t"
output = "codigos_estados.csv"  # troque para .xlsx se for Excel

# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_estados = pd.read_csv(output)
except Exception:
    df_estados = pd.read_csv(output, sep=';', encoding='utf-8', engine='python')

# Define as chaves dos estados
df_estados.sort_values(by=['NO_ESTADO'],inplace=True)
dicionario_estados = df_estados.set_index("NO_ESTADO")["CO_UF"].to_dict()
dicionario_estados[''] = None
dicionario_estados_inverse ={}
for chave, valor in dicionario_estados.items():
  dicionario_estados_inverse[valor]=chave

# Carrega dados dos códigos dos municipios
id_google = '1gXlpC4KysOh3inuQSNvgfecn3l_3wWjd'
url = f"https://drive.google.com/uc?id={id_google}&confirm=t"
output = "codigos_municipios.csv"  # troque para .xlsx se for Excel

# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_municipios = pd.read_csv(output)
except Exception:
    df_municipios = pd.read_csv(output, sep=';', encoding='utf-8', engine='python')

# Define as chaves dos municipios
df_municipios['NO_MUNICIPIO'] = df_municipios.apply(lambda x: titulo_formatado(x['NO_MUNICIPIO']), axis=1)
df_municipios.sort_values(by=['NO_MUNICIPIO'],inplace=True)
dicionario_municipios = df_municipios.set_index("NO_MUNICIPIO")["CO_MUNICIPIO"].to_dict()
dicionario_municipios[''] = None
dicionario_municipios_inverse ={}
for chave, valor in dicionario_municipios.items():
  dicionario_municipios_inverse[valor]=chave

# Carrega dados das Coortes
id_google = '1-5yaW6mP8IJM7khxOuAisbFhcCOOfNW6'
url = f"https://drive.google.com/uc?id={id_google}&confirm=t"
output = "indicadores_trajetoria_educacao_superior_2014_ate_2023.csv"  # troque para .xlsx se for Excel

# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_coorte = pd.read_csv(output)
except Exception:
    df_coorte = pd.read_csv(output, sep=';', encoding='utf-8', engine='python')

df_coorte['NO_CURSO'] = df_coorte.apply(lambda x: titulo_formatado(x['NO_CURSO']), axis=1)
df_coorte['NO_IES'] = df_coorte.apply(lambda x: titulo_formatado(x['NO_IES']), axis=1)


# Define os valores dos códigos de campos, seguindo o dicionário de dados
TP_CATEGORIA_ADMINISTRATIVA_KEYS = {
1: "Pública Federal",
2: "Pública Estadual",
3: "Pública Municipal",
4: "Privada com fins lucrativos",
5: "Privada sem fins lucrativos",
7: "Especial",
}
TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE ={}
for chave, valor in TP_CATEGORIA_ADMINISTRATIVA_KEYS.items():
  TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE[valor]=chave

TP_ORGANIZACAO_ACADEMICA_KEYS = {
1: "Universidade",
2: "Centro Universitário",
3: "Faculdade",
4: "Instituto Federal de Educação, Ciência e Tecnologia",
5: "Centro Federal de Educação Tecnológica",
}
TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE = {}
for chave, valor in TP_ORGANIZACAO_ACADEMICA_KEYS.items():
  TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE[valor]=chave

CO_REGIAO_KEYS = {
0: "Não Definida",
1: "Norte",
2: "Nordeste",
3: "Sudeste",
4: "Sul",
5: "Centro-Oeste",
}
CO_REGIAO_KEYS_INVERSE = {}
for chave, valor in CO_REGIAO_KEYS.items():
  CO_REGIAO_KEYS_INVERSE[valor]=chave

TP_GRAU_ACADEMICO_KEYS ={
1: "Bacharelado",
2: "Licenciatura",
3: "Tecnológico",
}
TP_GRAU_ACADEMICO_KEYS_INVERSE = {}
for chave, valor in TP_GRAU_ACADEMICO_KEYS.items():
  TP_GRAU_ACADEMICO_KEYS_INVERSE[valor]=chave

TP_MODALIDADE_ENSINO_KEYS ={
1: "Presencial",
2: "Curso a distância",
}
TP_MODALIDADE_ENSINO_KEYS_INVERSE ={}
for chave, valor in TP_MODALIDADE_ENSINO_KEYS.items():
  TP_MODALIDADE_ENSINO_KEYS_INVERSE[valor]=chave

# Carrega os dados
# df_coorte = pd.read_csv("/content/drive/MyDrive/PROGRAD/Censo/INEP/indicadores_trajetoria_educacao_superior_2014_ate_2023.csv")
# df_estados = pd.read_csv("/content/drive/MyDrive/PROGRAD/Censo/INEP/Dados_Estados_Municípios - Estados.csv")
# df_municipios = pd.read_csv("/content/drive/MyDrive/PROGRAD/Censo/INEP/Dados_Estados_Municípios - Município.csv")

anos = sorted(list(df_coorte['NU_ANO_INGRESSO'].unique()))

IES = sorted(df_coorte['NO_IES'].unique())
CURSOS = sorted(df_coorte['NO_CURSO'].unique())
CINE_ROTULO = sorted(df_coorte['NO_CINE_ROTULO'].unique())
CINE_AREA_GERAL = sorted(df_coorte['NO_CINE_AREA_GERAL'].unique())


# Carrega dados dos Cursos
id_google = '1yFPQlaMlFUOeChwgPdnnC57Qzb7Nukfy'
url = f"https://drive.google.com/uc?id={id_google}&confirm=t"
output = "cursos.xlsx"  # troque para .xlsx se for Excel

# Faz o download do arquivo
gdown.download(url, output, quiet=False)

# Tenta ler como CSV
try:
    df_cursos = pd.read_excel(output)
except Exception:
    print('Erro ao carregar arquivo')



df_coorte_ufc = df_coorte[df_coorte['NO_IES']==S_UFC]
df_cursos_ufc = df_coorte_ufc[['CO_CURSO','NO_CURSO','CO_MUNICIPIO','TP_GRAU_ACADEMICO']].drop_duplicates().copy()
df_cursos_ufc['GRAU'] = df_cursos_ufc['TP_GRAU_ACADEMICO'].map(TP_GRAU_ACADEMICO_KEYS)
df_cursos_ufc=df_cursos_ufc.merge(df_municipios,how='left', on='CO_MUNICIPIO' )
df_cursos_ufc['NO_MUNICIPIO'] = df_cursos_ufc['NO_MUNICIPIO'].fillna('EAD')
df_cursos_ufc['SG_ESTADO'] = df_cursos_ufc['SG_ESTADO'].fillna('CE')


df_cursos = df_cursos.merge(df_cursos_ufc[['CO_CURSO','NO_CURSO']], how='left',left_on='Código',right_on='CO_CURSO')
df_cursos['Curso'] = df_cursos.apply(lambda x: titulo_formatado(x['Curso']), axis=1)

# Unidades Acadêmicas
unidades_academicas = sorted(list(df_cursos['Unidade Acadêmica'].unique()))

# Retorna os códigos inep das unidades acadêmicas
def get_codigos_cursos(df_cursos, unidade_academica):
  return df_cursos[df_cursos['Unidade Acadêmica']==unidade_academica]['Código'].unique()


# Define os valores dos códigos de campos, seguindo o dicionário de dados
TP_CATEGORIA_ADMINISTRATIVA_KEYS = {
1: "Pública Federal",
2: "Pública Estadual",
3: "Pública Municipal",
4: "Privada com fins lucrativos",
5: "Privada sem fins lucrativos",
7: "Especial",
}
TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE ={}
for chave, valor in TP_CATEGORIA_ADMINISTRATIVA_KEYS.items():
  TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE[valor]=chave

TP_ORGANIZACAO_ACADEMICA_KEYS = {
1: "Universidade",
2: "Centro Universitário",
3: "Faculdade",
4: "Instituto Federal de Educação, Ciência e Tecnologia",
5: "Centro Federal de Educação Tecnológica",
}
TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE = {}
for chave, valor in TP_ORGANIZACAO_ACADEMICA_KEYS.items():
  TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE[valor]=chave

CO_REGIAO_KEYS = {
0: "Não Definida",
1: "Norte",
2: "Nordeste",
3: "Sudeste",
4: "Sul",
5: "Centro-Oeste",
}
CO_REGIAO_KEYS_INVERSE = {}
for chave, valor in CO_REGIAO_KEYS.items():
  CO_REGIAO_KEYS_INVERSE[valor]=chave

TP_GRAU_ACADEMICO_KEYS ={
1: "Bacharelado",
2: "Licenciatura",
3: "Tecnológico",
}
TP_GRAU_ACADEMICO_KEYS_INVERSE = {}
for chave, valor in TP_GRAU_ACADEMICO_KEYS.items():
  TP_GRAU_ACADEMICO_KEYS_INVERSE[valor]=chave

TP_MODALIDADE_ENSINO_KEYS ={
1: "Presencial",
2: "Curso a distância",
}
TP_MODALIDADE_ENSINO_KEYS_INVERSE ={}
for chave, valor in TP_MODALIDADE_ENSINO_KEYS.items():
  TP_MODALIDADE_ENSINO_KEYS_INVERSE[valor]=chave

anos = sorted(list(df_coorte['NU_ANO_INGRESSO'].unique()))

IES = sorted(df_coorte['NO_IES'].unique())
CURSOS = sorted(df_coorte['NO_CURSO'].unique())
CINE_ROTULO = sorted(df_coorte['NO_CINE_ROTULO'].unique())
CINE_AREA_GERAL = sorted(df_coorte['NO_CINE_AREA_GERAL'].unique())

# Calcula o ano de cada curso ao longo da coorte
df_coorte['ANO']= df_coorte['NU_ANO_REFERENCIA']-df_coorte['NU_ANO_INGRESSO']+1

df_coorte_IES = df_coorte[['CO_IES','NO_IES']].drop_duplicates()

# Calcula as coortes do ano de referência
df_coorte.sort_values(by=['CO_REGIAO','CO_UF','NO_IES','NO_CURSO','CO_CURSO',
                                                  'CO_MUNICIPIO',
                                                 'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO'
                                                 ,'NU_ANO_INGRESSO','NU_ANO_REFERENCIA'],inplace=True)
df_coorte_ano_referencia = df_coorte.groupby(by=['NO_IES','NO_CURSO','CO_CURSO',
                                                 'NO_CINE_ROTULO','NO_CINE_AREA_GERAL',
                                                 'TP_CATEGORIA_ADMINISTRATIVA','TP_ORGANIZACAO_ACADEMICA',
                                                 'CO_REGIAO','CO_UF', 'CO_MUNICIPIO',
                                                 'TP_GRAU_ACADEMICO', 'TP_MODALIDADE_ENSINO'
                                                 ,'NU_ANO_INGRESSO']).agg(
    NU_ANO_REFERENCIA=('NU_ANO_REFERENCIA','last'),
    QT_INGRESSANTE=('QT_INGRESSANTE','last'),
    QT_PERMANENCIA=('QT_PERMANENCIA','last'),
    QT_CONCLUINTES=('QT_CONCLUINTE', 'sum'),
    QT_DESISTENCIAS=('QT_DESISTENCIA','sum'),
    QT_FALECIDOS=('QT_FALECIDO','sum'),
    TAP=('TAP','last'),
    TCA=('TCA','last'),
    TDA=('TDA','last'),
    ).reset_index()

# df_coorte_ano_referencia

Downloading...
From: https://drive.google.com/uc?id=1qosijWRFUd32b_0erehxdXq2a-W5NYg5&confirm=t
To: /content/codigos_estados.csv
100%|██████████| 505/505 [00:00<00:00, 922kB/s]
Downloading...
From: https://drive.google.com/uc?id=1gXlpC4KysOh3inuQSNvgfecn3l_3wWjd&confirm=t
To: /content/codigos_municipios.csv
100%|██████████| 137k/137k [00:00<00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5yaW6mP8IJM7khxOuAisbFhcCOOfNW6&confirm=t
To: /content/indicadores_trajetoria_educacao_superior_2014_ate_2023.csv
100%|██████████| 314M/314M [00:02<00:00, 130MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yFPQlaMlFUOeChwgPdnnC57Qzb7Nukfy&confirm=t
To: /content/cursos.xlsx
100%|██████████| 14.9k/14.9k [00:00<00:00, 21.7MB/s]


In [3]:
#@title Coorte por ano
def coorte_grafico(df_coorte,ano_referencia=2023,anos=[2014, 2015, 2016], filtros={'NO_CURSO':'CIÊNCIA DA COMPUTAÇÃO', 'NO_IES':S_UFC, 'TP_MODALIDADE_ENSINO':1}, filename=None, fig_display=True):
  df_anos_ingresso=df_coorte[df_coorte["NU_ANO_INGRESSO"].isin(anos)]
  for chave, valor in filtros.items():
    if chave=='UNIDADE_ACADEMICA':
      df_anos_ingresso = df_anos_ingresso[df_anos_ingresso['CO_CURSO'].isin(get_codigos_cursos(df_cursos,valor))]
    else:
      df_anos_ingresso = df_anos_ingresso[df_anos_ingresso[chave]==valor]
  df_anos_ingresso = df_anos_ingresso.copy()

  # Filtrar coortes específicas
  df_agrupado = df_anos_ingresso.groupby(['NU_ANO_INGRESSO', 'NU_ANO_REFERENCIA'])[["TAP", "TCA", "TDA"]].mean().reset_index()
  if(len(df_agrupado)==0):
    print('Não há informação com os filtros selecionados')
    return

  # Pivotar dados
  df_pivot = df_agrupado.pivot(index="NU_ANO_REFERENCIA", columns="NU_ANO_INGRESSO", values=["TAP", "TCA", "TDA"])

  # 🟩 Verde (Conclusão)
  # #81C784 — Verde claro (suave, amigável)

  # #4CAF50 — Verde padrão (neutro e equilibrado)

  # #2E7D32 — Verde escuro (forte, confiável)

  # 🟥 Vermelho (Desistência)
  # #E57373 — Vermelho claro (menos agressivo)

  # #F44336 — Vermelho padrão (alerta, visível)

  # #C62828 — Vermelho escuro (impactante, forte)

  # 🟦 Azul (Permanência)
  # #64B5F6 — Azul claro (leve, calmo)

  # #2196F3 — Azul padrão (estável, confiável)

  # #1565C0 — Azul escuro (profundo, sólido)

  # Cores específicas por ano e categoria
  cores = {
      f"TDA_0": "#E57373",  # Desistência
      f"TCA_0": "#81C784",  # Conclusão
      f"TAP_0": "#64B5F6",  # Permanência

      f"TDA_1": "#F44336",
      f"TCA_1": "#4CAF50",
      f"TAP_1": "#2196F3",

      f"TDA_2": "#C62828",
      f"TCA_2": "#4B8B60",
      f"TAP_2": "#1565C0",
  }

  # Plotar gráfico
  fig, ax = plt.subplots(figsize=(15, 8))
  anos_ref = df_pivot.index
  if len(anos)==3:
    bar_width = 0.2
    deslocamento = {f"{anos[0]}": -0.25, f"{anos[1]}": 0, f"{anos[2]}": 0.25}
  elif len(anos)==2:
    bar_width = 0.3
    deslocamento = {f"{anos[0]}": -0.18, f"{anos[1]}": 0.18}
  elif len(anos)==1:
    bar_width = 0.4
    deslocamento = {f"{anos[0]}": 0}
  i = 0
  for ano in anos:
      tda = df_pivot["TDA"][ano].fillna(0)
      tca = df_pivot["TCA"][ano].fillna(0)
      tap = df_pivot["TAP"][ano].fillna(0)
      desloc = deslocamento[str(ano)]
      pos_x = anos_ref + desloc

      ax.bar(pos_x, tda, color=cores[f"TDA_{i}"], width=bar_width, label=f'Desistência ({ano})')
      ax.bar(pos_x, tca, bottom=tda, color=cores[f"TCA_{i}"], width=bar_width, label=f'Conclusão ({ano})')
      ax.bar(pos_x, tap, bottom=tda + tca, color=cores[f"TAP_{i}"], width=bar_width, label=f'Permanência ({ano})')

      i = i+1
      # Rótulos dentro das barras
      for x, y1, y2, y3 in zip(pos_x, tda, tca, tap):
          if y1 > 0:
              ax.text(x, y1 / 2, f"{y1:.1f}", ha='center', va='center', fontsize=9)
          if y2 > 0:
              ax.text(x, y1 + y2 / 2, f"{y2:.1f}", ha='center', va='center', fontsize=9)
          if y3 > 0:
              ax.text(x, y1 + y2 + y3 / 2, f"{y3:.1f}", ha='center', va='center', fontsize=9)

  # Rótulos e estética
  ax.set_xlabel("Ano de Referência", fontsize=12)
  ax.set_ylabel("Percentual (%)", fontsize=12)
  ax.set_xticks(anos_ref)
  ax.set_xticklabels(anos_ref, rotation=45, fontsize=12)

  if len(anos)==2:
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, fontsize=12, frameon=False)
  else:
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=12, frameon=False)
  ax.grid(axis='y', linestyle='--', alpha=0.7)
  s_title = f"Coortes {', '.join([f'{a}-{ano_referencia}' for a in anos])}"
  if 'NO_CURSO' in filtros:
    s_title = f"{s_title} \n{filtros['NO_CURSO']}"
    if 'CO_CURSO' in filtros:
      s_title = f"{s_title} ({filtros['CO_CURSO']})"
  if 'TP_GRAU_ACADEMICO' in filtros:
    s_title = f"{s_title} \nGrau: {TP_GRAU_ACADEMICO_KEYS[filtros['TP_GRAU_ACADEMICO']]}"
  if 'TP_MODALIDADE_ENSINO' in filtros:
    s_title = f"{s_title} \nModalidade: {TP_MODALIDADE_ENSINO_KEYS[filtros['TP_MODALIDADE_ENSINO']]}"
  if 'TP_ORGANIZACAO_ACADEMICA' in filtros:
    s_title = f"{s_title} \nOrganização: {TP_ORGANIZACAO_ACADEMICA_KEYS[filtros['TP_ORGANIZACAO_ACADEMICA']]}"
  if 'TP_CATEGORIA_ADMINISTRATIVA' in filtros:
    s_title = f"{s_title} \nCategoria: {TP_CATEGORIA_ADMINISTRATIVA_KEYS[filtros['TP_CATEGORIA_ADMINISTRATIVA']]}"
  if 'CO_REGIAO' in filtros:
    s_title = f"{s_title} \nRegião: {CO_REGIAO_KEYS[filtros['CO_REGIAO']]}"
  if 'NO_IES' in filtros:
    s_title = f"{s_title} \n{filtros['NO_IES']}"
  if 'NO_CINE_ROTULO' in filtros:
    s_title = f"{s_title} \nCine Rótulo: {filtros['NO_CINE_ROTULO']}"
  if 'NO_CINE_AREA_GERAL' in filtros:
    s_title = f"{s_title} \nCine Área Geral: {filtros['NO_CINE_AREA_GERAL']}"
  # if 'NO_CURSO' in filtros:
  #   s_title = f"{s_title} \n{filtros['NO_CURSO']}"
  #   if 'CO_CURSO' in filtros:
  #     s_title = f"{s_title} ({filtros['CO_CURSO']})"
  # if 'NO_IES' in filtros:
  #   s_title = f"{s_title} - {filtros['NO_IES']}"
  plt.title(s_title)
  plt.tight_layout()
  if filename:
    plt.savefig(f"{filename}.png", format="png", bbox_inches='tight')
  if fig_display:
    plt.show()
  plt.close()


def display_filtrar_coorte_ano():
  run = widgets.Button(description="Gerar")
  # anos_aux = [int(a) for a in anos]
  # print(anos_aux)
  wAno = widgets.Dropdown(
      options=anos,
      value=anos[0],
      description=f'Coorte Ano',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )
  wQtdAnos = widgets.Dropdown(
      options=[1,2,3],
      value=1,
      description=f'Qtd. de Coortes',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )

  wIES = widgets.Dropdown(
      options=['']+IES,
      value=S_UFC,
      description=f'IES',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )
  wUNIDADES_ACADEMICAS = widgets.Dropdown(
        options=['']+unidades_academicas,
        value=None,
        description=f'Unidade Acadêmica',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wCURSOS = widgets.Dropdown(
        options=['']+CURSOS,
        value=None,
        description=f'Curso',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wCINE_ROTULO = widgets.Dropdown(
        options=['']+CINE_ROTULO,
        value=None,
        description=f'CINE',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  wCINE_AREA_GERAL = widgets.Dropdown(
        options=['']+CINE_AREA_GERAL,
        value=None,
        description=f'CINE Área',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  TP_GRAU_ACADEMICO_KEYS_INVERSE['']=None
  wGRAU = widgets.Dropdown(
        options=TP_GRAU_ACADEMICO_KEYS_INVERSE,
        value=None,
        description=f'Grau',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE['']=None
  wCATEGORIA = widgets.Dropdown(
        options=TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE,
        value=None,
        description=f'Categoria',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE['']=None
  wORGANIZACAO = widgets.Dropdown(
        options=TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE,
        value=None,
        description=f'Organização',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  TP_MODALIDADE_ENSINO_KEYS_INVERSE['']=None
  wMODALIDADE = widgets.Dropdown(
        options=TP_MODALIDADE_ENSINO_KEYS_INVERSE,
        value=None,
        description=f'Modalidade',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  # CO_REGIAO_KEYS_INVERSE
  CO_REGIAO_KEYS_INVERSE['']=None
  wREGIAO = widgets.Dropdown(
        options=CO_REGIAO_KEYS_INVERSE,
        value=None,
        description=f'Região',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  # CO_ESTADO_KEYS_INVERSE
  wESTADO = widgets.Dropdown(
        options=dicionario_estados,
        value=None,
        description=f'Estado',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wMUNICIPIO = widgets.Dropdown(
        options=dicionario_municipios,
        value=None,
        description=f'Município',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )


  wCodigoCursos = widgets.Textarea(
        description=f'Cód. Curso',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False

  )

  output = widgets.Output()
  display(wAno, wQtdAnos, wORGANIZACAO, wCATEGORIA,
          wREGIAO,wESTADO,wMUNICIPIO,
          wIES,
          wGRAU,wMODALIDADE,
          wUNIDADES_ACADEMICAS,
          wCURSOS,wCodigoCursos,
          wCINE_AREA_GERAL,wCINE_ROTULO,run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      erro = False
      filtros=dict()
      if wORGANIZACAO.value:
        filtros['TP_ORGANIZACAO_ACADEMICA']=wORGANIZACAO.value
      if wCATEGORIA.value:
        filtros['TP_CATEGORIA_ADMINISTRATIVA']=wCATEGORIA.value
      if wREGIAO.value:
        filtros['CO_REGIAO']=wREGIAO.value
      if wESTADO.value:
        filtros['CO_UF']=wESTADO.value
      if wMUNICIPIO.value:
        filtros['CO_MUNICIPIO']=wMUNICIPIO.value
      if wIES.value:
        filtros['NO_IES']=wIES.value
      if wCURSOS.value:
        filtros['NO_CURSO']=wCURSOS.value
      if wUNIDADES_ACADEMICAS.value:
        filtros['UNIDADE_ACADEMICA']=wUNIDADES_ACADEMICAS.value
      if wGRAU.value:
        filtros['TP_GRAU_ACADEMICO']=wGRAU.value
      if wMODALIDADE.value:
        filtros['TP_MODALIDADE_ENSINO']=wMODALIDADE.value
      if wCINE_AREA_GERAL.value:
        filtros['NO_CINE_AREA_GERAL']=wCINE_AREA_GERAL.value
      if wCINE_ROTULO.value:
        filtros['NO_CINE_ROTULO']=wCINE_ROTULO.value
      if wCodigoCursos.value:
        filtros['CO_CURSO']=int(wCodigoCursos.value)
      anos_coortes = [wAno.value+i for i in range(wQtdAnos.value) if wAno.value+i in anos ]
      coorte_grafico(df_coorte,anos=anos_coortes
               , filtros=filtros)

  run.on_click(on_button_run_clicked)

display_filtrar_coorte_ano()


Dropdown(description='Coorte Ano', layout=Layout(width='600px'), options=(np.int64(2014), np.int64(2015), np.i…

Dropdown(description='Qtd. de Coortes', layout=Layout(width='600px'), options=(1, 2, 3), style=DescriptionStyl…

Dropdown(description='Organização', layout=Layout(width='600px'), options={'Universidade': 1, 'Centro Universi…

Dropdown(description='Categoria', layout=Layout(width='600px'), options={'Pública Federal': 1, 'Pública Estadu…

Dropdown(description='Região', layout=Layout(width='600px'), options={'Não Definida': 0, 'Norte': 1, 'Nordeste…

...

Textarea(value='', description='Cód. Curso', layout=Layout(width='600px'), style=DescriptionStyle(description_…

Dropdown(description='CINE Área', layout=Layout(width='600px'), options=('', 'Agricultura, silvicultura, pesca…

Dropdown(description='CINE', layout=Layout(width='600px'), options=('', 'Administração', 'Administração públic…

Button(description='Gerar', style=ButtonStyle())

Output()

In [4]:
#@title Coorte por ano relativo de ingresso
def coorte_grafico_ingresso(df_coorte,ano_referencia=2023,anos=[2014, 2015, 2016], filtros={'NO_CURSO':'CIÊNCIA DA COMPUTAÇÃO', 'NO_IES':S_UFC, 'TP_MODALIDADE_ENSINO':1}, filename=None, fig_display=True):
  df_anos_ingresso=df_coorte[df_coorte["NU_ANO_INGRESSO"].isin(anos)]
  for chave, valor in filtros.items():
    if chave=='UNIDADE_ACADEMICA':
      df_anos_ingresso = df_anos_ingresso[df_anos_ingresso['CO_CURSO'].isin(get_codigos_cursos(df_cursos,valor))]
    else:
      df_anos_ingresso = df_anos_ingresso[df_anos_ingresso[chave]==valor]
  df_anos_ingresso = df_anos_ingresso.copy()

  df_anos_ingresso['ANO_RELATIVO'] = df_anos_ingresso['NU_ANO_REFERENCIA']-df_anos_ingresso['NU_ANO_INGRESSO']+1
  # Filtrar coortes específicas
  df_agrupado = df_anos_ingresso.groupby(['NU_ANO_INGRESSO','ANO_RELATIVO'])[["TAP", "TCA", "TDA"]].mean().reset_index()
  if(len(df_agrupado)==0):
    print('Não há informação com os filtros selecionados')
    return

  # Pivotar dados
  df_pivot = df_agrupado.pivot(index="ANO_RELATIVO", columns="NU_ANO_INGRESSO", values=["TAP", "TCA", "TDA"])

  # 🟩 Verde (Conclusão)
  # #81C784 — Verde claro (suave, amigável)

  # #4CAF50 — Verde padrão (neutro e equilibrado)

  # #2E7D32 — Verde escuro (forte, confiável)

  # 🟥 Vermelho (Desistência)
  # #E57373 — Vermelho claro (menos agressivo)

  # #F44336 — Vermelho padrão (alerta, visível)

  # #C62828 — Vermelho escuro (impactante, forte)

  # 🟦 Azul (Permanência)
  # #64B5F6 — Azul claro (leve, calmo)

  # #2196F3 — Azul padrão (estável, confiável)

  # #1565C0 — Azul escuro (profundo, sólido)

  # Cores específicas por ano e categoria
  cores = {
      f"TDA_0": "#E57373",  # Desistência
      f"TCA_0": "#81C784",  # Conclusão
      f"TAP_0": "#64B5F6",  # Permanência

      f"TDA_1": "#F44336",
      f"TCA_1": "#4CAF50",
      f"TAP_1": "#2196F3",

      f"TDA_2": "#C62828",
      f"TCA_2": "#4B8B60",
      f"TAP_2": "#1565C0",
  }

  # Plotar gráfico
  fig, ax = plt.subplots(figsize=(15, 8))
  anos_ref = df_pivot.index
  if len(anos)==3:
    bar_width = 0.2
    deslocamento = {f"{anos[0]}": -0.25, f"{anos[1]}": 0, f"{anos[2]}": 0.25}
  elif len(anos)==2:
    bar_width = 0.3
    deslocamento = {f"{anos[0]}": -0.18, f"{anos[1]}": 0.18}
  elif len(anos)==1:
    bar_width = 0.4
    deslocamento = {f"{anos[0]}": 0}
  i = 0
  for ano in anos:
      tda = df_pivot["TDA"][ano].fillna(0)
      tca = df_pivot["TCA"][ano].fillna(0)
      tap = df_pivot["TAP"][ano].fillna(0)
      desloc = deslocamento[str(ano)]
      pos_x = anos_ref + desloc

      ax.bar(pos_x, tda, color=cores[f"TDA_{i}"], width=bar_width, label=f'Desistência ({ano})')
      ax.bar(pos_x, tca, bottom=tda, color=cores[f"TCA_{i}"], width=bar_width, label=f'Conclusão ({ano})')
      ax.bar(pos_x, tap, bottom=tda + tca, color=cores[f"TAP_{i}"], width=bar_width, label=f'Permanência ({ano})')

      i = i+1
      # Rótulos dentro das barras
      for x, y1, y2, y3 in zip(pos_x, tda, tca, tap):
          if y1 > 0:
              ax.text(x, y1 / 2, f"{y1:.1f}", ha='center', va='center', fontsize=9)
          if y2 > 0:
              ax.text(x, y1 + y2 / 2, f"{y2:.1f}", ha='center', va='center', fontsize=9)
          if y3 > 0:
              ax.text(x, y1 + y2 + y3 / 2, f"{y3:.1f}", ha='center', va='center', fontsize=9)

  # Rótulos e estética
  ax.set_xlabel("Ano Relativo ao Ingresso", fontsize=12)
  ax.set_ylabel("Percentual (%)", fontsize=12)
  ax.set_xticks(anos_ref)
  ax.set_xticklabels(anos_ref, fontsize=12)
  # ax.set_xticklabels(anos_ref, rotation=45)
  if len(anos)==2:
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, fontsize=12, frameon=False)
  else:
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=12, frameon=False)
  ax.grid(axis='y', linestyle='--', alpha=0.7)
  s_title = f"Coortes {', '.join([f'{a}-{ano_referencia}' for a in anos])}"
  if 'NO_CURSO' in filtros:
    s_title = f"{s_title} \n{filtros['NO_CURSO']}"
    if 'CO_CURSO' in filtros:
      s_title = f"{s_title} ({filtros['CO_CURSO']})"
  if 'TP_GRAU_ACADEMICO' in filtros:
    s_title = f"{s_title} \nGrau: {TP_GRAU_ACADEMICO_KEYS[filtros['TP_GRAU_ACADEMICO']]}"
  if 'TP_MODALIDADE_ENSINO' in filtros:
    s_title = f"{s_title} \nModalidade: {TP_MODALIDADE_ENSINO_KEYS[filtros['TP_MODALIDADE_ENSINO']]}"
  if 'TP_ORGANIZACAO_ACADEMICA' in filtros:
    s_title = f"{s_title} \nOrganização: {TP_ORGANIZACAO_ACADEMICA_KEYS[filtros['TP_ORGANIZACAO_ACADEMICA']]}"
  if 'TP_CATEGORIA_ADMINISTRATIVA' in filtros:
    s_title = f"{s_title} \nCategoria: {TP_CATEGORIA_ADMINISTRATIVA_KEYS[filtros['TP_CATEGORIA_ADMINISTRATIVA']]}"
  if 'CO_REGIAO' in filtros:
    s_title = f"{s_title} \nRegião: {CO_REGIAO_KEYS[filtros['CO_REGIAO']]}"
  if 'NO_IES' in filtros:
    s_title = f"{s_title} \n{filtros['NO_IES']}"
  if 'NO_CINE_ROTULO' in filtros:
    s_title = f"{s_title} \nCine Rótulo: {filtros['NO_CINE_ROTULO']}"
  if 'NO_CINE_AREA_GERAL' in filtros:
    s_title = f"{s_title} \nCine Área Geral: {filtros['NO_CINE_AREA_GERAL']}"
  # if 'NO_CURSO' in filtros:
  #   s_title = f"{s_title} \n{filtros['NO_CURSO']}"
  #   if 'CO_CURSO' in filtros:
  #     s_title = f"{s_title} ({filtros['CO_CURSO']})"
  # if 'NO_IES' in filtros:
  #   s_title = f"{s_title} - {filtros['NO_IES']}"
  plt.title(s_title)
  plt.tight_layout()
  if filename:
    plt.savefig(f"{filename}.png", format="png", bbox_inches='tight')
  if fig_display:
    plt.show()
  plt.close()


def display_filtrar_coorte_ano_ingresso():
  run = widgets.Button(description="Gerar")
  # anos_aux = [int(a) for a in anos]
  # print(anos_aux)
  wAno = widgets.Dropdown(
      options=anos,
      value=anos[0],
      description=f'Coorte Ano',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )
  wQtdAnos = widgets.Dropdown(
      options=[1,2,3],
      value=1,
      description=f'Qtd. de Coortes',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )

  wIES = widgets.Dropdown(
      options=['']+IES,
      value=S_UFC,
      description=f'IES',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )
  wUNIDADES_ACADEMICAS = widgets.Dropdown(
        options=['']+unidades_academicas,
        value=None,
        description=f'Unidade Acadêmica',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wCURSOS = widgets.Dropdown(
        options=['']+CURSOS,
        value=None,
        description=f'Curso',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wCINE_ROTULO = widgets.Dropdown(
        options=['']+CINE_ROTULO,
        value=None,
        description=f'CINE',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  wCINE_AREA_GERAL = widgets.Dropdown(
        options=['']+CINE_AREA_GERAL,
        value=None,
        description=f'CINE Área',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  TP_GRAU_ACADEMICO_KEYS_INVERSE['']=None
  wGRAU = widgets.Dropdown(
        options=TP_GRAU_ACADEMICO_KEYS_INVERSE,
        value=None,
        description=f'Grau',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE['']=None
  wCATEGORIA = widgets.Dropdown(
        options=TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE,
        value=None,
        description=f'Categoria',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE['']=None
  wORGANIZACAO = widgets.Dropdown(
        options=TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE,
        value=None,
        description=f'Organização',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  TP_MODALIDADE_ENSINO_KEYS_INVERSE['']=None
  wMODALIDADE = widgets.Dropdown(
        options=TP_MODALIDADE_ENSINO_KEYS_INVERSE,
        value=None,
        description=f'Modalidade',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  # CO_REGIAO_KEYS_INVERSE
  CO_REGIAO_KEYS_INVERSE['']=None
  wREGIAO = widgets.Dropdown(
        options=CO_REGIAO_KEYS_INVERSE,
        value=None,
        description=f'Região',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  # CO_ESTADO_KEYS_INVERSE
  wESTADO = widgets.Dropdown(
        options=dicionario_estados,
        value=None,
        description=f'Estado',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wMUNICIPIO = widgets.Dropdown(
        options=dicionario_municipios,
        value=None,
        description=f'Município',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )


  wCodigoCursos = widgets.Textarea(
        description=f'Cód. Curso',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False

  )

  output = widgets.Output()
  display(wAno, wQtdAnos, wORGANIZACAO, wCATEGORIA,
          wREGIAO,wESTADO,wMUNICIPIO,
          wIES,
          wGRAU,wMODALIDADE,
          wUNIDADES_ACADEMICAS,
          wCURSOS,wCodigoCursos,
          wCINE_AREA_GERAL,wCINE_ROTULO,run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      erro = False
      filtros=dict()
      if wORGANIZACAO.value:
        filtros['TP_ORGANIZACAO_ACADEMICA']=wORGANIZACAO.value
      if wCATEGORIA.value:
        filtros['TP_CATEGORIA_ADMINISTRATIVA']=wCATEGORIA.value
      if wREGIAO.value:
        filtros['CO_REGIAO']=wREGIAO.value
      if wESTADO.value:
        filtros['CO_UF']=wESTADO.value
      if wMUNICIPIO.value:
        filtros['CO_MUNICIPIO']=wMUNICIPIO.value
      if wIES.value:
        filtros['NO_IES']=wIES.value
      if wCURSOS.value:
        filtros['NO_CURSO']=wCURSOS.value
      if wUNIDADES_ACADEMICAS.value:
        filtros['UNIDADE_ACADEMICA']=wUNIDADES_ACADEMICAS.value
      if wGRAU.value:
        filtros['TP_GRAU_ACADEMICO']=wGRAU.value
      if wMODALIDADE.value:
        filtros['TP_MODALIDADE_ENSINO']=wMODALIDADE.value
      if wCINE_AREA_GERAL.value:
        filtros['NO_CINE_AREA_GERAL']=wCINE_AREA_GERAL.value
      if wCINE_ROTULO.value:
        filtros['NO_CINE_ROTULO']=wCINE_ROTULO.value
      if wCodigoCursos.value:
        filtros['CO_CURSO']=int(wCodigoCursos.value)
      anos_coortes = [wAno.value+i for i in range(wQtdAnos.value) if wAno.value+i in anos ]
      coorte_grafico_ingresso(df_coorte,anos=anos_coortes
               , filtros=filtros)

  run.on_click(on_button_run_clicked)

display_filtrar_coorte_ano_ingresso()


Dropdown(description='Coorte Ano', layout=Layout(width='600px'), options=(np.int64(2014), np.int64(2015), np.i…

Dropdown(description='Qtd. de Coortes', layout=Layout(width='600px'), options=(1, 2, 3), style=DescriptionStyl…

Dropdown(description='Organização', layout=Layout(width='600px'), options={'Universidade': 1, 'Centro Universi…

Dropdown(description='Categoria', layout=Layout(width='600px'), options={'Pública Federal': 1, 'Pública Estadu…

Dropdown(description='Região', layout=Layout(width='600px'), options={'Não Definida': 0, 'Norte': 1, 'Nordeste…

...

Textarea(value='', description='Cód. Curso', layout=Layout(width='600px'), style=DescriptionStyle(description_…

Dropdown(description='CINE Área', layout=Layout(width='600px'), options=('', 'Agricultura, silvicultura, pesca…

Dropdown(description='CINE', layout=Layout(width='600px'), options=('', 'Administração', 'Administração públic…

Button(description='Gerar', style=ButtonStyle())

Output()

In [6]:
#@title Coortes
# Função que gera os gráficos das coortes
def coortes_grafico(df_coorte_ano_referencia, ano_referencia=2023, filtros={'NO_CURSO':'CIÊNCIA DA COMPUTAÇÃO', 'NO_IES':S_UFC, 'TP_MODALIDADE_ENSINO':1}, filename=None, fig_display=True):
  df_filtrado = df_coorte_ano_referencia
  for chave, valor in filtros.items():
    if chave=='UNIDADE_ACADEMICA':
      df_filtrado = df_filtrado[df_filtrado['CO_CURSO'].isin(get_codigos_cursos(df_cursos,valor))]
    else:
      df_filtrado = df_filtrado[df_filtrado[chave]==valor]
  # Cores específicas por ano e categoria
  cores = {
      f"TDA_0": "#E57373",  # Desistência
      f"TCA_0": "#81C784",  # Conclusão
      f"TAP_0": "#64B5F6",  # Permanência
  }


  # Filtrar coortes específicas
  df_agrupado = df_filtrado.groupby(['NU_ANO_INGRESSO', 'NU_ANO_REFERENCIA'])[["TAP", "TCA", "TDA"]].mean().reset_index()
  df_agrupado_anos_refs = df_agrupado.groupby(by=['NU_ANO_INGRESSO']).nth(-1)[['NU_ANO_INGRESSO','NU_ANO_REFERENCIA']]
  anos_refs = dict(zip(df_agrupado_anos_refs['NU_ANO_INGRESSO'], df_agrupado_anos_refs['NU_ANO_REFERENCIA']))
  if(len(df_agrupado)==0):
    print('Não há informação com os filtros selecionados')
    return
  # Pivotar dados
  df_pivot = df_agrupado.pivot(index="NU_ANO_INGRESSO", columns="NU_ANO_REFERENCIA", values=["TAP", "TCA", "TDA"])

  # fig, ax = plt.subplots()
  fig, ax = plt.subplots(figsize=(10, 8))
  anos_ref = df_pivot.index
  bar_width = 0.3
  deslocamento = {f"{anos[0]}": -0.25, f"{anos[1]}": 0, f"{anos[2]}": 0.25}
  i = 0
  tda = df_pivot["TDA"][ano_referencia]
  tca = df_pivot["TCA"][ano_referencia]
  tap = df_pivot["TAP"][ano_referencia]
  pos_x = anos_ref

  ax.bar(pos_x, tda, color=cores[f"TDA_{i}"], width=bar_width, label=f'Desistência')
  ax.bar(pos_x, tca, bottom=tda, color=cores[f"TCA_{i}"], width=bar_width, label=f'Conclusão')
  ax.bar(pos_x, tap, bottom=tda + tca, color=cores[f"TAP_{i}"], width=bar_width, label=f'Permanência')

  i = i+1
  # Rótulos dentro das barras
  for x, y1, y2, y3 in zip(pos_x, tda, tca, tap):
      if y1 > 0:
          ax.text(x, y1 / 2, f"{y1:.1f}", ha='center', va='center', fontsize=9)
      if y2 > 0:
          ax.text(x, y1 + y2 / 2, f"{y2:.1f}", ha='center', va='center', fontsize=9)
      if y3 > 0:
          ax.text(x, y1 + y2 + y3 / 2, f"{y3:.1f}", ha='center', va='center', fontsize=9)

  # Rótulos e estética
  # ax.set_xlabel("Ano de Ingresso")
  ax.set_ylabel("Percentual (%)")
  ax.set_xticks(anos_ref)
  # ax.set_xticklabels(anos_ref, rotation=45)
  ax.set_xticklabels([f'{a}-{ano_referencia}' for a in anos_ref], rotation=45)

  ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=12, frameon=False)
  ax.grid(axis='y', linestyle='--', alpha=0.7)
  # s_title = f"Coortes - Ano Referência {ano}"
  s_title = f"Coortes {', '.join([f'{a}-{anos_refs[a]}' for a in anos if a in anos_refs.keys()])}"

  if 'NO_CURSO' in filtros:
    s_title = f"{s_title} \n{filtros['NO_CURSO']}"
    if 'CO_CURSO' in filtros:
      s_title = f"{s_title} ({filtros['CO_CURSO']})"
  if 'TP_GRAU_ACADEMICO' in filtros:
    s_title = f"{s_title} \nGrau: {TP_GRAU_ACADEMICO_KEYS[filtros['TP_GRAU_ACADEMICO']]}"
  if 'TP_MODALIDADE_ENSINO' in filtros:
    s_title = f"{s_title} \nModalidade: {TP_MODALIDADE_ENSINO_KEYS[filtros['TP_MODALIDADE_ENSINO']]}"
  if 'TP_ORGANIZACAO_ACADEMICA' in filtros:
    s_title = f"{s_title} \nOrganização: {TP_ORGANIZACAO_ACADEMICA_KEYS[filtros['TP_ORGANIZACAO_ACADEMICA']]}"
  if 'TP_CATEGORIA_ADMINISTRATIVA' in filtros:
    s_title = f"{s_title} \nCategoria: {TP_CATEGORIA_ADMINISTRATIVA_KEYS[filtros['TP_CATEGORIA_ADMINISTRATIVA']]}"
  if 'CO_REGIAO' in filtros:
    s_title = f"{s_title} \nRegião: {CO_REGIAO_KEYS[filtros['CO_REGIAO']]}"
  if 'CO_UF' in filtros:
    s_title = f"{s_title} \nEstado: {dicionario_estados_inverse[filtros['CO_UF']]}"
  if 'CO_MUNICIPIO' in filtros:
    s_title = f"{s_title} \Município: {dicionario_municipios_inverse[filtros['CO_MUNICIPIO']]}"
  if 'UNIDADE_ACADEMICA' in filtros:
    s_title = f"{s_title} \nUnidade Acadêmica: {filtros['UNIDADE_ACADEMICA']}"
  if 'NO_IES' in filtros:
    s_title = f"{s_title} \n{filtros['NO_IES']}"
  if 'NO_CINE_ROTULO' in filtros:
    s_title = f"{s_title} \nCine Rótulo: {filtros['NO_CINE_ROTULO']}"
  if 'NO_CINE_AREA_GERAL' in filtros:
    s_title = f"{s_title} \nCine Área Geral: {filtros['NO_CINE_AREA_GERAL']}"

  plt.title(s_title)
  plt.tight_layout()
  if filename:
    plt.savefig(f"{filename}.png", format="png", bbox_inches='tight')
  if fig_display:
    plt.show()
  plt.close()


def display_filtrar_coortes():
  run = widgets.Button(description="Gerar")
  wIES = widgets.Dropdown(
      options=['']+IES,
      value=S_UFC,
      description=f'IES',
      style={'description_width': '150px'},
      layout=widgets.Layout(width='600px'),
      disabled=False
      )
  wUNIDADES_ACADEMICAS = widgets.Dropdown(
        options=['']+unidades_academicas,
        value=None,
        description=f'Unidade Acadêmica',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wCURSOS = widgets.Dropdown(
        options=['']+CURSOS,
        value=None,
        description=f'Curso',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wCINE_ROTULO = widgets.Dropdown(
        options=['']+CINE_ROTULO,
        value=None,
        description=f'CINE',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  wCINE_AREA_GERAL = widgets.Dropdown(
        options=['']+CINE_AREA_GERAL,
        value=None,
        description=f'CINE Área',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  TP_GRAU_ACADEMICO_KEYS_INVERSE['']=None
  wGRAU = widgets.Dropdown(
        options=TP_GRAU_ACADEMICO_KEYS_INVERSE,
        value=None,
        description=f'Grau',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE['']=None
  wCATEGORIA = widgets.Dropdown(
        options=TP_CATEGORIA_ADMINISTRATIVA_KEYS_INVERSE,
        value=None,
        description=f'Categoria',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE['']=None
  wORGANIZACAO = widgets.Dropdown(
        options=TP_ORGANIZACAO_ACADEMICA_KEYS_INVERSE,
        value=None,
        description=f'Organização',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  TP_MODALIDADE_ENSINO_KEYS_INVERSE['']=None
  wMODALIDADE = widgets.Dropdown(
        options=TP_MODALIDADE_ENSINO_KEYS_INVERSE,
        value=None,
        description=f'Modalidade',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )
  # CO_REGIAO_KEYS_INVERSE
  CO_REGIAO_KEYS_INVERSE['']=None
  wREGIAO = widgets.Dropdown(
        options=CO_REGIAO_KEYS_INVERSE,
        value=None,
        description=f'Região',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  # CO_ESTADO_KEYS_INVERSE
  wESTADO = widgets.Dropdown(
        options=dicionario_estados,
        value=None,
        description=f'Estado',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )

  wMUNICIPIO = widgets.Dropdown(
        options=dicionario_municipios,
        value=None,
        description=f'Município',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False
        )


  wCodigoCursos = widgets.Textarea(
        description=f'Cód. Curso',
        style={'description_width': '150px'},
        layout=widgets.Layout(width='600px'),
        disabled=False

  )

  output = widgets.Output()
  display(wORGANIZACAO, wCATEGORIA,
          wREGIAO,wESTADO,wMUNICIPIO,
          wIES,
          wGRAU,wMODALIDADE,
          wUNIDADES_ACADEMICAS,
          wCURSOS,wCodigoCursos,
          wCINE_AREA_GERAL,wCINE_ROTULO,run,output)

  def on_button_run_clicked(_):
    output.clear_output()
    with output:
      erro = False
      filtros=dict()
      if wORGANIZACAO.value:
        filtros['TP_ORGANIZACAO_ACADEMICA']=wORGANIZACAO.value
      if wCATEGORIA.value:
        filtros['TP_CATEGORIA_ADMINISTRATIVA']=wCATEGORIA.value
      if wREGIAO.value:
        filtros['CO_REGIAO']=wREGIAO.value
      if wESTADO.value:
        filtros['CO_UF']=wESTADO.value
      if wMUNICIPIO.value:
        filtros['CO_MUNICIPIO']=wMUNICIPIO.value
      if wIES.value:
        filtros['NO_IES']=wIES.value
      if wCURSOS.value:
        filtros['NO_CURSO']=wCURSOS.value
      if wUNIDADES_ACADEMICAS.value:
        filtros['UNIDADE_ACADEMICA']=wUNIDADES_ACADEMICAS.value
      if wGRAU.value:
        filtros['TP_GRAU_ACADEMICO']=wGRAU.value
      if wMODALIDADE.value:
        filtros['TP_MODALIDADE_ENSINO']=wMODALIDADE.value
      if wCINE_AREA_GERAL.value:
        filtros['NO_CINE_AREA_GERAL']=wCINE_AREA_GERAL.value
      if wCINE_ROTULO.value:
        filtros['NO_CINE_ROTULO']=wCINE_ROTULO.value
      if wCodigoCursos.value:
        filtros['CO_CURSO']=int(wCodigoCursos.value)
      coortes_grafico(df_coorte_ano_referencia,ano_referencia=2023, filtros=filtros)

  run.on_click(on_button_run_clicked)

display_filtrar_coortes()

Dropdown(description='Organização', layout=Layout(width='600px'), options={'Universidade': 1, 'Centro Universi…

Dropdown(description='Categoria', layout=Layout(width='600px'), options={'Pública Federal': 1, 'Pública Estadu…

Dropdown(description='Região', layout=Layout(width='600px'), options={'Não Definida': 0, 'Norte': 1, 'Nordeste…

Dropdown(description='Estado', layout=Layout(width='600px'), options={'Acre': 12, 'Alagoas': 27, 'Amapa': 16, …

Dropdown(description='Município', layout=Layout(width='600px'), options={'Abadia de Goias': 5200050, 'Abadia d…

...

Textarea(value='', description='Cód. Curso', layout=Layout(width='600px'), style=DescriptionStyle(description_…

Dropdown(description='CINE Área', layout=Layout(width='600px'), options=('', 'Agricultura, silvicultura, pesca…

Dropdown(description='CINE', layout=Layout(width='600px'), options=('', 'Administração', 'Administração públic…

Button(description='Gerar', style=ButtonStyle())

Output()